In [1]:
import pandas as pd
import numpy as np

df1=pd.read_csv('Dataset.csv')
df2=pd.read_csv('Movie_Id_Titles.csv')

In [2]:
df1.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [3]:
df2.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [4]:
df1.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [5]:
df2.isnull().sum()

item_id    0
title      0
dtype: int64

In [6]:
df1=df1.drop(['timestamp'],axis='columns')
df1.head()

,user_id,item_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [7]:
df=pd.merge(df1,df2,left_on='item_id',right_on='item_id')
df

,user_id,item_id,rating,title
0,0,50,5,Star Wars (1977)
1,290,50,5,Star Wars (1977)
2,79,50,4,Star Wars (1977)
3,2,50,5,Star Wars (1977)
4,8,50,5,Star Wars (1977)
...,...,...,...,...
99998,840,1674,4,Mamma Roma (1962)
99999,655,1640,3,"Eighth Day, The (1996)"
100000,655,1637,3,Girls Town (1996)
100001,655,1630,3,"Silence of the Palace, The (Saimt el Qusur) (1..."


In [8]:
df['item_id'].nunique()

1682

In [9]:
df['user_id'].nunique()

944

In [10]:
df['title'].nunique()

1664

In [11]:
df['rating'].unique()

array([5, 4, 2, 3, 1], dtype=int64)

In [12]:
def convert_rating(rating):
    if rating >= 4:
        return 1
    else:
        return 0

In [13]:
mov_list=df.groupby(['user_id','title'])['rating'].sum().unstack().reset_index().fillna(0).set_index("user_id")

In [15]:
!pip install mlxtend

  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/73/da/d5d77a9a7a135c948dbf8d3b873655b105a152d69e590150c83d23c3d070/mlxtend-0.23.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.4 MB 409.6 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.4 MB 409.6 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.4 MB 652.5 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.4 MB 655.4 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.4 MB 689.9 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.4 MB 689.9 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 628.5 kB/s eta 0:00:02
   -------- ------------------------------- 0.3

In [16]:
from mlxtend.frequent_patterns import apriori,association_rules

In [17]:
mov_list = mov_list.applymap(convert_rating)

In [18]:
frequent_movi = apriori(mov_list , use_colnames=True ,min_support=0.08)
frequent_movi

C:\Users\bhuva\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.115466,(12 Angry Men (1957))
1,0.198093,(2001: A Space Odyssey (1968))
2,0.090042,"(Abyss, The (1989))"
3,0.131356,"(African Queen, The (1951))"
4,0.266949,(Air Force One (1997))
...,...,...
14256,0.080508,"(Indiana Jones and the Last Crusade (1989), Em..."
14257,0.081568,"(Princess Bride, The (1987), Empire Strikes Ba..."
14258,0.080508,"(Terminator, The (1984), Empire Strikes Back, ..."
14259,0.082627,"(Terminator, The (1984), Indiana Jones and the..."


In [19]:
result = association_rules(frequent_movi , metric='lift' , min_threshold=1)
result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(12 Angry Men (1957)),(Raiders of the Lost Ark (1981)),0.115466,0.368644,0.080508,0.697248,1.891385,0.037943,2.085388,0.532808
1,(Raiders of the Lost Ark (1981)),(12 Angry Men (1957)),0.368644,0.115466,0.080508,0.218391,1.891385,0.037943,1.131683,0.746468
2,(12 Angry Men (1957)),"(Silence of the Lambs, The (1991))",0.115466,0.364407,0.082627,0.715596,1.963729,0.040550,2.234828,0.554829
3,"(Silence of the Lambs, The (1991))",(12 Angry Men (1957)),0.364407,0.115466,0.082627,0.226744,1.963729,0.040550,1.143909,0.772137
4,(12 Angry Men (1957)),(Star Wars (1977)),0.115466,0.531780,0.092161,0.798165,1.500932,0.030758,2.319819,0.377314
...,...,...,...,...,...,...,...,...,...,...
166889,(Terminator 2: Judgment Day (1991)),"(Terminator, The (1984), Empire Strikes Back, ...",0.239407,0.100636,0.080508,0.336283,3.341593,0.056416,1.355042,0.921309
166890,"(Silence of the Lambs, The (1991))","(Terminator, The (1984), Empire Strikes Back, ...",0.364407,0.104873,0.080508,0.220930,2.106648,0.042292,1.148969,0.826491
166891,(Star Wars (1977)),"(Terminator, The (1984), Empire Strikes Back, ...",0.531780,0.082627,0.080508,0.151394,1.832261,0.036569,1.081036,0.970112
166892,(Raiders of the Lost Ark (1981)),"(Terminator, The (1984), Empire Strikes Back, ...",0.368644,0.081568,0.080508,0.218391,2.677415,0.050439,1.175053,0.992317


In [20]:
fin_result=result[['antecedents','consequents','support','confidence','lift','leverage']]
fin_result

,antecedents,consequents,support,confidence,lift,leverage
0,(12 Angry Men (1957)),(Raiders of the Lost Ark (1981)),0.080508,0.697248,1.891385,0.037943
1,(Raiders of the Lost Ark (1981)),(12 Angry Men (1957)),0.080508,0.218391,1.891385,0.037943
2,(12 Angry Men (1957)),"(Silence of the Lambs, The (1991))",0.082627,0.715596,1.963729,0.040550
3,"(Silence of the Lambs, The (1991))",(12 Angry Men (1957)),0.082627,0.226744,1.963729,0.040550
4,(12 Angry Men (1957)),(Star Wars (1977)),0.092161,0.798165,1.500932,0.030758
...,...,...,...,...,...,...
166889,(Terminator 2: Judgment Day (1991)),"(Terminator, The (1984), Empire Strikes Back, ...",0.080508,0.336283,3.341593,0.056416
166890,"(Silence of the Lambs, The (1991))","(Terminator, The (1984), Empire Strikes Back, ...",0.080508,0.220930,2.106648,0.042292
166891,(Star Wars (1977)),"(Terminator, The (1984), Empire Strikes Back, ...",0.080508,0.151394,1.832261,0.036569
166892,(Raiders of the Lost Ark (1981)),"(Terminator, The (1984), Empire Strikes Back, ...",0.080508,0.218391,2.677415,0.050439


In [21]:
pred_1=association_rules(frequent_movi,metric='support',min_threshold=0.05)
pred_1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(12 Angry Men (1957)),(Raiders of the Lost Ark (1981)),0.115466,0.368644,0.080508,0.697248,1.891385,0.037943,2.085388,0.532808
1,(Raiders of the Lost Ark (1981)),(12 Angry Men (1957)),0.368644,0.115466,0.080508,0.218391,1.891385,0.037943,1.131683,0.746468
2,(12 Angry Men (1957)),"(Silence of the Lambs, The (1991))",0.115466,0.364407,0.082627,0.715596,1.963729,0.040550,2.234828,0.554829
3,"(Silence of the Lambs, The (1991))",(12 Angry Men (1957)),0.364407,0.115466,0.082627,0.226744,1.963729,0.040550,1.143909,0.772137
4,(12 Angry Men (1957)),(Star Wars (1977)),0.115466,0.531780,0.092161,0.798165,1.500932,0.030758,2.319819,0.377314
...,...,...,...,...,...,...,...,...,...,...
166953,(Terminator 2: Judgment Day (1991)),"(Terminator, The (1984), Empire Strikes Back, ...",0.239407,0.100636,0.080508,0.336283,3.341593,0.056416,1.355042,0.921309
166954,"(Silence of the Lambs, The (1991))","(Terminator, The (1984), Empire Strikes Back, ...",0.364407,0.104873,0.080508,0.220930,2.106648,0.042292,1.148969,0.826491
166955,(Star Wars (1977)),"(Terminator, The (1984), Empire Strikes Back, ...",0.531780,0.082627,0.080508,0.151394,1.832261,0.036569,1.081036,0.970112
166956,(Raiders of the Lost Ark (1981)),"(Terminator, The (1984), Empire Strikes Back, ...",0.368644,0.081568,0.080508,0.218391,2.677415,0.050439,1.175053,0.992317


In [22]:
pred_1[['antecedents','consequents','support','confidence','lift','leverage']].to_csv('pred_1_invoice.csv')

In [25]:
dfr=pd.read_csv('pred_1_invoice.csv',index_col=0)
dfr

,antecedents,consequents,support,confidence,lift,leverage
0,frozenset({'12 Angry Men (1957)'}),frozenset({'Raiders of the Lost Ark (1981)'}),0.080508,0.697248,1.891385,0.037943
1,frozenset({'Raiders of the Lost Ark (1981)'}),frozenset({'12 Angry Men (1957)'}),0.080508,0.218391,1.891385,0.037943
2,frozenset({'12 Angry Men (1957)'}),"frozenset({'Silence of the Lambs, The (1991)'})",0.082627,0.715596,1.963729,0.040550
3,"frozenset({'Silence of the Lambs, The (1991)'})",frozenset({'12 Angry Men (1957)'}),0.082627,0.226744,1.963729,0.040550
4,frozenset({'12 Angry Men (1957)'}),frozenset({'Star Wars (1977)'}),0.092161,0.798165,1.500932,0.030758
...,...,...,...,...,...,...
166953,"frozenset({'Silence of the Lambs, The (1991)'})","frozenset({'Empire Strikes Back, The (1980)', ...",0.080508,0.220930,2.106648,0.042292
166954,"frozenset({'Terminator, The (1984)'})","frozenset({'Empire Strikes Back, The (1980)', ...",0.080508,0.350230,3.983344,0.060297
166955,frozenset({'Raiders of the Lost Ark (1981)'}),"frozenset({'Empire Strikes Back, The (1980)', ...",0.080508,0.218391,2.677415,0.050439
166956,frozenset({'Terminator 2: Judgment Day (1991)'}),"frozenset({'Empire Strikes Back, The (1980)', ...",0.080508,0.336283,3.341593,0.056416


In [26]:
dfr['antecedents']=dfr['antecedents'].str.replace('frozenset','')
dfr['consequents']=dfr['consequents'].str.replace('frozenset','')

In [27]:
dfr.head()

,antecedents,consequents,support,confidence,lift,leverage
0,({'12 Angry Men (1957)'}),({'Raiders of the Lost Ark (1981)'}),0.080508,0.697248,1.891385,0.037943
1,({'Raiders of the Lost Ark (1981)'}),({'12 Angry Men (1957)'}),0.080508,0.218391,1.891385,0.037943
2,({'12 Angry Men (1957)'}),"({'Silence of the Lambs, The (1991)'})",0.082627,0.715596,1.963729,0.040550
3,"({'Silence of the Lambs, The (1991)'})",({'12 Angry Men (1957)'}),0.082627,0.226744,1.963729,0.040550
4,({'12 Angry Men (1957)'}),({'Star Wars (1977)'}),0.092161,0.798165,1.500932,0.030758


In [28]:
title="({'Star Wars (1977)'})"
dfr[dfr['antecedents']==title]

,antecedents,consequents,support,confidence,lift,leverage
5,({'Star Wars (1977)'}),({'12 Angry Men (1957)'}),0.092161,0.173307,1.500932,0.030758
83,({'Star Wars (1977)'}),({'2001: A Space Odyssey (1968)'}),0.154661,0.290837,1.468181,0.049319
115,({'Star Wars (1977)'}),"({'African Queen, The (1951)'})",0.109110,0.205179,1.562010,0.039258
143,({'Star Wars (1977)'}),({'Air Force One (1997)'}),0.128178,0.241036,0.902928,-0.013780
189,({'Star Wars (1977)'}),({'Aladdin (1992)'}),0.126059,0.237052,1.554006,0.044940
...,...,...,...,...,...,...
166449,({'Star Wars (1977)'}),"({'Empire Strikes Back, The (1980)', 'Back to ...",0.080508,0.151394,1.856056,0.037132
166575,({'Star Wars (1977)'}),"({'Empire Strikes Back, The (1980)', 'Princess...",0.081568,0.153386,1.880478,0.038192
166700,({'Star Wars (1977)'}),"({'Empire Strikes Back, The (1980)', 'Fugitive...",0.080508,0.151394,1.856056,0.037132
166826,({'Star Wars (1977)'}),"({'Empire Strikes Back, The (1980)', 'Indiana ...",0.082627,0.155378,1.856675,0.038124


In [29]:
print(dfr[dfr['consequents'] == title].antecedents)

4                                 ({'12 Angry Men (1957)'})
82                       ({'2001: A Space Odyssey (1968)'})
114                         ({'African Queen, The (1951)'})
142                              ({'Air Force One (1997)'})
188                                    ({'Aladdin (1992)'})
                                ...                        
166332    ({'Empire Strikes Back, The (1980)', 'Back to ...
166458    ({'Empire Strikes Back, The (1980)', 'Princess...
166585    ({'Empire Strikes Back, The (1980)', 'Fugitive...
166711    ({'Empire Strikes Back, The (1980)', 'Indiana ...
166837    ({'Empire Strikes Back, The (1980)', 'Silence ...
Name: antecedents, Length: 5329, dtype: object
